In [154]:
import sys
import pandas as pd
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from recommenders.models.ncf.ncf_singlenode import NCF
from recommenders.models.ncf.dataset import Dataset as NCFDataset
from recommenders.datasets.python_splitters import python_stratified_split
from recommenders.utils.constants import SEED
from recommenders.evaluation.python_evaluation import (
    ndcg_at_k, precision_at_k, recall_at_k
)
from collections import defaultdict
import os
import numpy as np
import tensorflow as tf
import tf_slim as slim
from time import time
import logging

tf.compat.v1.disable_eager_execution()
logger = logging.getLogger(__name__)
MODEL_CHECKPOINT = "model.ckpt"

In [67]:
# top k items to recommend
TOP_K = 5



# Model parameters
EPOCHS = 100
BATCH_SIZE = 256

In [108]:
ratings = pd.read_csv('books_data/ratings.csv', sep=';', error_bad_lines=False, encoding="latin-1")
books = pd.read_csv('books_data/books.csv', sep=';', error_bad_lines=False, encoding="latin-1")

C:\Users\hamza\AppData\Local\Temp\ipykernel_23224\2762359282.py:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  ratings = pd.read_csv('books_data/ratings.csv', sep=';', error_bad_lines=False, encoding="latin-1")
C:\Users\hamza\AppData\Local\Temp\ipykernel_23224\2762359282.py:2: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  books = pd.read_csv('books_data/books.csv', sep=';', error_bad_lines=False, encoding="latin-1")
Skipping line 6452: expected 8 fields, saw 9
Skipping line 43667: expected 8 fields, saw 10
Skipping line 51751: expected 8 fields, saw 9

Skipping line 92038: expected 8 fields, saw 9
Skipping line 104319: expected 8 fields, saw 9
Skipping line 121768: expected 8 fields, saw 9

Skipping line 144058: expected 8 fields, saw 9
Skipping line 150789: expected 8 fields, saw 9
Skipping line 

In [128]:
ratings_explicit = ratings[ratings['Book-Rating'] != 0]
books_df= pd.merge(books, ratings_explicit, on='ISBN')
books_df.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L,User-ID,Book-Rating
0,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,8,5
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,11676,8
2,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,67544,8
3,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,116866,9
4,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,123629,9


In [129]:
books_df.rename(columns = {'Book-Title':'title', 'Book-Author' : 'author', 'User-ID' : 'userID', 'Book-Rating' : 'rating','ISBN' : 'itemID'  }, inplace = True) 
books_df.drop(['Image-URL-S', 'Image-URL-M', 'Image-URL-L', 'Year-Of-Publication', 'Publisher'], axis= 1, inplace=True)
books_df = books_df[['userID', 'itemID', 'rating', 'title', 'author']]
books_df.head()

,userID,itemID,rating,title,author
0,8,0002005018,5,Clara Callan,Richard Bruce Wright
1,11676,0002005018,8,Clara Callan,Richard Bruce Wright
2,67544,0002005018,8,Clara Callan,Richard Bruce Wright
3,116866,0002005018,9,Clara Callan,Richard Bruce Wright
4,123629,0002005018,9,Clara Callan,Richard Bruce Wright


In [130]:
user_interaction_counts = books_df.groupby('userID')['itemID'].nunique()
users_with_x_interaction = user_interaction_counts[user_interaction_counts > 20]
len(users_with_x_interaction)

3138

In [132]:
item_interaction_count = books_df.groupby('itemID')['userID'].nunique()
items_with_x_interaction = item_interaction_count[item_interaction_count > 1]
len(items_with_x_interaction)

50423

In [133]:
books_df = books_df[books_df['userID'].isin(users_with_x_interaction.index)]
books_df = books_df[books_df['itemID'].isin(items_with_x_interaction.index)]


In [134]:
n_books = books_df['itemID'].nunique()
n_users = books_df['userID'].nunique()
n_ratings = books_df.shape[0]


print(f"Number of ratings: {n_ratings}")
print(f"Number of unique books's: {n_books}")
print(f"Number of unique users: {n_users}")
print(f"Average number of ratings per user: {round(n_ratings/n_users, 2)}")
print(f"Average number of ratings per book: {round(n_ratings/n_books, 2)}")


Number of ratings: 151450
Number of unique books's: 44430
Number of unique users: 3138
Average number of ratings per user: 48.26
Average number of ratings per book: 3.41


In [135]:
unique_userIds = books_df['userID'].unique()
unique_userIds.sort()

# Create a mapping from old userIds to new ones (starting from 1)
user_id_mapping = {old_id: new_id for new_id, old_id in enumerate(unique_userIds, start=1)}

# Apply the mapping to the userIds column
books_df['userID'] = books_df['userID'].map(user_id_mapping)
books_df.sort_values(by='userID', ascending=True)

,userID,itemID,rating,title,author
5005,1,3257233051,9,Veronika Deschliesst Zu Sterben / Vernika Deci...,Paolo Coelho
5044,1,3596292646,7,Das Buch der lÃ?Â¤cherlichen Liebe.,Milan Kundera
5039,1,3499110695,7,Neun ErzÃ?Â¤hlungen.,Jerome D. Salinger
5034,1,3498020862,8,Die Korrekturen.,Jonathan Franzen
5025,1,3492045642,8,QuerschÃ?Â¼sse - Downsize This!,Michael Moore
...,...,...,...,...,...
69036,3138,0515131083,10,Plantation: A Lowcountry Tale,Dorothea Benton Frank
27665,3138,0671038184,8,Jewel,Bret Lott
313345,3138,0425178900,8,Stick Figure: A Diary of My Former Self,Lori Gottlieb
106513,3138,0375703063,7,A Virtuous Woman (Oprah's Book Club (Paperback)),Kaye Gibbons


In [136]:
unique_itemIds = books_df['itemID'].unique()

# Create a mapping from old itemIdsunique_itemIds to new ones (starting from 1)
item_id_mapping = {old_id: new_id for new_id, old_id in enumerate(unique_itemIds, start=1)}

# Apply the mapping to the itemIdsunique_itemIds column
books_df['itemID'] = books_df['itemID'].map(item_id_mapping)
books_df.sort_values(by='itemID', ascending=True)

,userID,itemID,rating,title,author
1,121,1,8,Clara Callan,Richard Bruce Wright
3,1313,1,9,Clara Callan,Richard Bruce Wright
7,2460,1,7,Clara Callan,Richard Bruce Wright
8,2964,1,6,Clara Callan,Richard Bruce Wright
11,384,2,6,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata
...,...,...,...,...,...
382472,2716,44428,4,Behavior Modification--What It Is and How to D...,Garry Martin
383170,2713,44429,8,Captives of the Night,Loretta Chase
383171,2757,44429,8,Captives of the Night,Loretta Chase
383678,1242,44430,9,Secret Windows: Essays and Fiction on the Craf...,Stephen King


In [137]:
temp = "./temp.csv"
books_df.to_csv(temp, index=False)



In [138]:
train, test = python_stratified_split(books_df)

In [92]:
leave_one_out_test = test.groupby("userID").last().reset_index()

In [139]:
test = test[test["userID"].isin(train["userID"].unique())]
test = test[test["itemID"].isin(train["itemID"].unique())]


In [140]:
print(f"Test set length: {test.shape[0]}")
print(f"Full data length: {train.shape[0] + test.shape[0]}")

Test set length: 31569
Full data length: 145172


In [141]:
train_file = "./train.csv"
test_file = "./test.csv"
leave_one_out_test_file = "./leave_one_out_test.csv"
train.to_csv(train_file, index=False)
test.to_csv(test_file, index=False)
leave_one_out_test.to_csv(leave_one_out_test_file, index=False)

In [142]:
data = NCFDataset(train_file=train_file, test_file=leave_one_out_test_file,binary = False, seed=SEED, overwrite_test_file_full=True)

INFO:recommenders.models.ncf.dataset:Indexing ./train.csv ...
INFO:recommenders.models.ncf.dataset:Indexing ./leave_one_out_test.csv ...
INFO:recommenders.models.ncf.dataset:Creating full leave-one-out test file ./leave_one_out_test_full.csv ...
100%|██████████| 991/991 [00:28<00:00, 34.67it/s]
INFO:recommenders.models.ncf.dataset:Indexing ./leave_one_out_test_full.csv ...


In [158]:
class CustomNCF(NCF):
    def _create_model(self):
        super()._create_model()
        # Override the loss function here
        with tf.compat.v1.variable_scope("loss", reuse=tf.compat.v1.AUTO_REUSE):
            mse_loss = tf.reduce_mean(tf.square(self.labels - self.output))
            self.loss = tf.sqrt(mse_loss)


In [156]:
model = CustomNCF(
    n_users=data.n_users, 
    n_items=data.n_items,
    model_type="GMF",
    n_factors=4,
    layer_sizes=[16,8,4],
    n_epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    learning_rate=1e-3,
    verbose=5,
    seed=SEED
)

In [157]:
model.fit(data)
model.save(dir_name=".pretrain/GMF")

INFO:recommenders.models.ncf.ncf_singlenode:Epoch 5 [79.27s]: train_loss = 3.258492 
INFO:recommenders.models.ncf.ncf_singlenode:Epoch 10 [75.75s]: train_loss = 3.220448 
INFO:recommenders.models.ncf.ncf_singlenode:Epoch 15 [61.47s]: train_loss = 3.209110 
INFO:recommenders.models.ncf.ncf_singlenode:Epoch 20 [60.62s]: train_loss = 3.202929 
INFO:recommenders.models.ncf.ncf_singlenode:Epoch 25 [61.49s]: train_loss = 3.197980 
INFO:recommenders.models.ncf.ncf_singlenode:Epoch 30 [77.20s]: train_loss = 3.194807 
INFO:recommenders.models.ncf.ncf_singlenode:Epoch 35 [59.05s]: train_loss = 3.192852 
INFO:recommenders.models.ncf.ncf_singlenode:Epoch 40 [60.64s]: train_loss = 3.191873 
INFO:recommenders.models.ncf.ncf_singlenode:Epoch 45 [59.71s]: train_loss = 3.191583 
INFO:recommenders.models.ncf.ncf_singlenode:Epoch 50 [59.32s]: train_loss = 3.190422 
INFO:recommenders.models.ncf.ncf_singlenode:Epoch 55 [60.91s]: train_loss = 3.190434 
INFO:recommenders.models.ncf.ncf_singlenode:Epoch 60 [6

In [159]:
model = CustomNCF(
    n_users=data.n_users, 
    n_items=data.n_items,
    model_type="MLP",
    n_factors=4,
    layer_sizes=[16,8,4],
    n_epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    learning_rate=1e-3,
    verbose=5,
    seed=SEED
)

c:\Users\hamza\anaconda3\envs\ncf_env\lib\site-packages\tensorflow\python\keras\engine\base_layer_v1.py:1694: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '


In [160]:
model.fit(data)
model.save(dir_name=".pretrain/MLP")

INFO:recommenders.models.ncf.ncf_singlenode:Epoch 5 [169.44s]: train_loss = 3.256353 
INFO:recommenders.models.ncf.ncf_singlenode:Epoch 10 [118.96s]: train_loss = 3.253336 
INFO:recommenders.models.ncf.ncf_singlenode:Epoch 15 [93.28s]: train_loss = 3.252209 


KeyboardInterrupt: 

In [ ]:
model = NCF(
    n_users=data.n_users, 
    n_items=data.n_items,
    model_type="NeuMF",
    n_factors=4,
    layer_sizes=[16,8,4],
    n_epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    learning_rate=1e-3,
    verbose=10,
    seed=SEED
)

model.load(gmf_dir=".pretrain/GMF", mlp_dir=".pretrain/MLP", alpha=0.5)

In [ ]:
model.fit(data)